In [1]:
from datasets import load_from_disk, load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import numpy as np
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, AutoModelForMaskedLM, BertConfig, BertModel, BertForPreTraining, BertForMaskedLM, AutoModel, PretrainedConfig, AutoConfig

import sys 
import os
sys.path.append(os.path.abspath(".."))

from collate import DataCollatorForLanguageModelingSpan


dataset = load_from_disk("/s/project/semi_supervised_multispecies/all_fungi_reference/fungi/Annotation/Sequences/AAA_Concatenated/pretokenized/speciesdownstream300")
dataset = dataset.train_test_split(test_size=0.1)
dataset = dataset.remove_columns(["species_name", "__index_level_0__"])


tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")

# This way we don't load weights
# https://stackoverflow.com/questions/65072694/make-sure-bert-model-does-not-load-pretrained-weights
# TODO AutConfig or AutoModel? i guess it doesn't matter
specieslm = AutoModelForMaskedLM.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")
speciesm2 = BertForMaskedLM.from_pretrained("../slurm/results/mp/checkpoint-4000", trust_remote_code=True)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [12]:
module = next(iter(speciesm2.modules()))
type(module.bert.encoder.layer[0].attention)

transformers.models.bert.modeling_bert.BertAttention

In [ ]:
specieslm.eval()
data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)
%env WANDB_PROJECT=singlesamplednam2

trainingArguments = TrainingArguments(   
    output_dir="eval_res",
    run_name="2batch64_monarchhf",
    report_to="wandb",
    eval_accumulation_steps=16,
)

trainer = Trainer(
    args=trainingArguments,
    model=specieslm,
    tokenizer=tokenizer,
    data_collator=data_collator
)

predictions = trainer.predict(dataset["test"])

env: WANDB_PROJECT=singlesamplednam2


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
%env WANDB_PROJECT=singlesamplednam2

data_collator = DataCollatorForLanguageModelingSpan(tokenizer, mlm=True, mlm_probability = 0.02, span_length = 6)

training_args = TrainingArguments(
    output_dir="./results",
    
    max_steps=20000,
    
    seed=17,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    
    logging_strategy="steps",
    logging_steps=1,
    
    evaluation_strategy="no",
    
    #dataloader_num_workers=4,
    #dataloader_prefetch_factor=2,
    run_name="2batch64_monarchhf",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)

trainer.train()

env: WANDB_PROJECT=singlesamplednam2


/opt/modules/i12g/anaconda/envs/m2-mixer/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luluhu (luluh). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,10.480400
2,10.433300
3,10.342700
4,10.326000
5,10.266300
6,10.228300
7,10.225500
8,10.166900
9,10.122700
10,10.069800


In [5]:
# Load model directly
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("togethercomputer/m2-bert-80M-2k")

You are using a model of type m2_bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


ValueError: Unrecognized configuration class <class 'transformers_modules.togethercomputer.m2-bert-80M-2k.3e061b6d18dd7fab5df5047432edca801fe89e5d.configuration_bert.BertConfig'> for this kind of AutoModel: AutoModelForMaskedLM.
Model type should be one of AlbertConfig, BartConfig, BertConfig, BigBirdConfig, CamembertConfig, ConvBertConfig, Data2VecTextConfig, DebertaConfig, DebertaV2Config, DistilBertConfig, ElectraConfig, ErnieConfig, EsmConfig, FlaubertConfig, FNetConfig, FunnelConfig, IBertConfig, LayoutLMConfig, LongformerConfig, LukeConfig, MBartConfig, MegaConfig, MegatronBertConfig, MobileBertConfig, MPNetConfig, MraConfig, MvpConfig, NezhaConfig, NystromformerConfig, PerceiverConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, SqueezeBertConfig, TapasConfig, Wav2Vec2Config, XLMConfig, XLMRobertaConfig, XLMRobertaXLConfig, XmodConfig, YosoConfig.